In [1]:
!pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 25.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Dense
import keras_tuner as kf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

Using TensorFlow backend


In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/aishwaryamate/Datasets/main/Churn_Modelling.csv')

In [4]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
df.drop(columns = ['RowNumber'	,'CustomerId',	'Surname'], inplace=True)

In [6]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [7]:
df = pd.get_dummies(data = df,columns=['Geography','Gender'])
df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,1,0,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,1,0,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,1,1,0,0,1,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1,0,0,1


In [8]:
x = df.drop(columns = ['Exited'])
y = df['Exited']

In [9]:
sc = StandardScaler()

x = sc.fit_transform(x)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size = 0.20, random_state=1)

In [12]:
ann = Sequential()

ann.add(Dense(units = 10, activation = 'relu'))
ann.add(Dense(units = 1, activation = 'sigmoid'))

ann.compile(optimizer = 'adadelta', loss = 'binary_crossentropy', metrics ='accuracy')

ann.fit(xtrain,ytrain,epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.7557 - accuracy: 0.5157
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.7546 - accuracy: 0.5167
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.7536 - accuracy: 0.5175
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.7525 - accuracy: 0.5186
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.7514 - accuracy: 0.5191
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.7502 - accuracy: 0.5200
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.7491 - accuracy: 0.5219
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.7479 - accuracy: 0.5226
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.7468 - accuracy: 0.5232
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.7457 - accura

In [14]:
ypred = ann.predict(xtest)
ypred = ypred > 0.5
ypred

63/63 [==============================] - 0s 2ms/step


array([[False],
       [False],
       [False],
       ...,
       [ True],
       [False],
       [ True]])

In [15]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.76      0.71      0.73      1585
           1       0.13      0.17      0.15       415

    accuracy                           0.59      2000
   macro avg       0.45      0.44      0.44      2000
weighted avg       0.63      0.59      0.61      2000



In [16]:
ann.evaluate(xtrain,ytrain)

250/250 [==============================] - 1s 2ms/step - loss: 0.6594 - accuracy: 0.6202


[0.6594085097312927, 0.6202499866485596]

# Hyperparameter tuning


In [18]:
counter = 0

for i in range(0,11):
  if counter == 0:
    print('Python')
else:
    print('Machine learning')
counter = counter+1

Python
Python
Python
Python
Python
Python
Python
Python
Python
Python
Python
Machine learning


In [19]:
counter

1

In [20]:
def hyper(hp):
    model = Sequential()
    counter = 0
    for i in range(hp.Int('num_layers', min_value = 1,max_value=10)):
        if counter ==0:
            model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                            activation = hp.Choice('activation'+str(i),values = ['relu','tanh','sigmoid']),
                            input_dim = 13))
    else:
        model.add(Dense(
              hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
              activation = hp.Choice('activation'+str(i),values = ['relu','tanh','sigmoid'])))
    counter+=1
    model.add(Dense(units = 1,activation='sigmoid'))
    model.compile(optimizer = hp.Choice('optimizer',values = ['adam','rmsprop','sgd']),
                loss = 'binary_crossentropy',metrics = 'accuracy')
    return model

In [21]:
tuner = kf.RandomSearch(hyper,
                        objective = 'val_accuracy',
                        max_trials = 3)

In [22]:
tuner.search(xtrain,ytrain,epochs = 5, validation_data = (xtest,ytest))

Trial 3 Complete [00h 00m 08s]
val_accuracy: 0.8659999966621399

Best val_accuracy So Far: 0.8659999966621399
Total elapsed time: 00h 00m 22s


In [23]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 10,
 'units0': 128,
 'activation0': 'relu',
 'optimizer': 'rmsprop',
 'units1': 16,
 'activation1': 'tanh',
 'units2': 32,
 'activation2': 'relu',
 'units3': 80,
 'activation3': 'sigmoid',
 'units4': 120,
 'activation4': 'relu',
 'units5': 32,
 'activation5': 'tanh',
 'units6': 96,
 'activation6': 'tanh',
 'units7': 64,
 'activation7': 'tanh',
 'units8': 56,
 'activation8': 'tanh',
 'units9': 40,
 'activation9': 'relu'}

In [25]:
df.shape

(10000, 14)

In [26]:
tuned_model = tuner.get_best_models(num_models=1)[0]
tuned_model.build(x.shape)

In [27]:
tuned_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1792      
                                                                 
 dense_1 (Dense)             (None, 16)                2064      
                                                                 
 dense_2 (Dense)             (None, 32)                544       
                                                                 
 dense_3 (Dense)             (None, 80)                2640      
                                                                 
 dense_4 (Dense)             (None, 120)               9720      
                                                                 
 dense_5 (Dense)             (None, 32)                3872      
                                                                 
 dense_6 (Dense)             (None, 96)                3

In [28]:
tuned_model.fit(xtrain,ytrain,epochs=50, validation_data=(xtest,ytest))

Epoch 1/50
250/250 [==============================] - 4s 6ms/step - loss: 0.3476 - accuracy: 0.8604 - val_loss: 0.3747 - val_accuracy: 0.8530
Epoch 2/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3427 - accuracy: 0.8590 - val_loss: 0.3485 - val_accuracy: 0.8610
Epoch 3/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3396 - accuracy: 0.8601 - val_loss: 0.3447 - val_accuracy: 0.8580
Epoch 4/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3396 - accuracy: 0.8645 - val_loss: 0.3755 - val_accuracy: 0.8560
Epoch 5/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3386 - accuracy: 0.8566 - val_loss: 0.3427 - val_accuracy: 0.8645
Epoch 6/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3383 - accuracy: 0.8599 - val_loss: 0.3397 - val_accuracy: 0.8565
Epoch 7/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3327 - accuracy: 0.8622 - val_loss: 0.3502 - val_accuracy: 0.8545
Epoch 

In [29]:
tuned_model.evaluate(xtrain,ytrain)

250/250 [==============================] - 1s 3ms/step - loss: 0.2651 - accuracy: 0.8876


[0.2651313245296478, 0.887624979019165]

In [30]:
tuned_model.evaluate(xtest,ytest)

63/63 [==============================] - 0s 2ms/step - loss: 0.4161 - accuracy: 0.8540


[0.41608044505119324, 0.8539999723434448]